In [1]:
%matplotlib inline
import os
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns

In [2]:
def correlation_study(df, station1, station2, parameter, isplot=1):
    newdf=df[['Station', 'sn', parameter]]
#     print(parameter)
#     print(newdf[newdf['Station']==station1].groupby('sn').mean().reset_index()[parameter])
    gradient, intercept, r_value, p_value, std_err = stats.linregress(newdf[newdf['Station']==station1].groupby('sn').mean().reset_index()[parameter], 
                                                                  newdf[newdf['Station']==station2].groupby('sn').mean().reset_index()[parameter])
#     print('Linear regression using stats.linregress')
#     print('regression: a=%.2f b=%.2f, std error= %.3f' % (gradient, intercept, std_err))
    if isplot:
        p = np.poly1d(np.array([gradient, intercept]))
        p(newdf[newdf['Station']==station1].groupby('sn').mean().reset_index()[parameter])
        plt.plot(newdf[newdf['Station']==station1].groupby('sn').mean(), 
                 newdf[newdf['Station']==station2].groupby('sn').mean(), 'o')
        plt.plot(newdf[newdf['Station']==station1].groupby('sn').mean().reset_index()[parameter],
                p(newdf[newdf['Station']==station1].groupby('sn').mean().reset_index()[parameter]))
        plt.title(parameter+'_correlation '+' m='+str(round(gradient,3))+' b='+str(round(intercept,3))+' R^2='+str(round(r_value**2, 3)))
        plt.xlabel(station1)
        plt.ylabel(station2)
        plt.grid()
        plt.savefig(station1+'-'+station2+'_'+df.Phase.unique()[0]+'_'+parameter+'.png', dpi=240)
        plt.close()
    return [df.Phase.unique()[0], round(gradient, 3), round(intercept, 3), round(r_value**2, 3), round(std_err, 3)]

In [9]:
def calculation(df, station1, station2):
    outdf=pd.DataFrame()
    for parameter in df.columns[4:]:
#         print parameter
        result = correlation_study(df, station1, station2, parameter, isplot=1)
#         print [station1, station2,  result[0], parameter, result[1], result[2], result[3], result[4]]
        testdf = pd.DataFrame({'Station1':[station1], 'Station2':[station2], 'Phase':[result[0]], 
                             'Parameter':[parameter], 'Slope':[result[1]], 'Intercept':[result[2]], 
                             'R_square':[result[3]], 'MSE':[result[4]]})
        outdf = outdf.append(testdf, ignore_index = True)

        columnsTitles = ['Station1', 'Station2', 'Phase', 'Parameter', 'Slope', 'Intercept', 'R_square', 'MSE']
    outdf = outdf.reindex(columns=columnsTitles)
    outdf = outdf.sort_values(by=['Parameter'])
    outdf.to_csv(station1+'-'+station2+'_'+result[0]+'_correlation.csv', index=False)

In [51]:
# read dataframe
df1 = pd.read_csv(r'C:\Users\fanchang\fan_python\G501_FGEO_02\G501_FGEO_02_phase_03_out.csv')
# df2 = pd.read_csv(r'C:\Users\fanchang\fan_python\FGEO_G601_02\FGEO_G601_02_phase_03_out.csv')
df3 = pd.read_csv(r'C:\Users\fanchang\fan_python\G501_FGEO_03\G501_FGEO_03_phase_03_out.csv')
df4 = pd.read_csv(r'C:\Users\fanchang\fan_python\G502_FGEO_01\G502_FGEO_01_phase_03_out.csv')
df5 = pd.read_csv(r'C:\Users\fanchang\fan_python\G502_FGEO_03\G502_FGEO_03_phase_03_out.csv')
# df6 = pd.read_csv(r'/home/fan/Downloads/G502_COMBO3_N3/G502_COMBO3_N3_phase_12_out.csv')
# df7 = pd.read_csv(os.path.join(folder, files[6])
# df8 = pd.read_csv(os.path.join(folder, files[7]))

# reformat dataframe to drop index
# df = pd.concat([df1, df2, df3, df4, df5, df6]).reset_index()
df = pd.concat([df1, df3, df4, df5]).reset_index()
df = df[df.columns[1:]]

# stations = [name.split('.')[0] for name in files]
stations =['G501_FGEO_02',  'G501_FGEO_03', 'G502_FGEO_01', 'G502_FGEO_03']
fullname = stations
for idx in range(len(fullname)):
    for idy in range(len(fullname)):
        if idx < idy:
            print fullname[idx], fullname[idy]
            calculation(df, fullname[idx], fullname[idy])

G501_FGEO_02 G501_FGEO_03
G501_FGEO_02 G502_FGEO_01
G501_FGEO_02 G502_FGEO_03
G501_FGEO_03 G502_FGEO_01
G501_FGEO_03 G502_FGEO_03
G502_FGEO_01 G502_FGEO_03


In [ ]:
station1 = 'G501_COMBO3_N1'
station2 = 'G501_COMBO3_N2'
parameter = 'SFR_Center_V'
correlation_study(df, station1, station2, parameter)

In [ ]:
station1 = 'G501_COMBO3_N1'
station2 = 'G502_COMBO3_N1'

outdf=pd.DataFrame()
for parameter in df.columns[4:]:
    result = correlation_study(df, station1, station2, parameter, isplot=1)
    print [station1, station2,  result[0], parameter, result[1], result[2], result[3], result[4]]
    testdf = pd.DataFrame({'Station1':[station1], 'Station2':[station2], 'Phase':[result[0]], 
                         'Parameter':[parameter], 'Slope':[result[1]], 'Intercept':[result[2]], 
                         'R_square':[result[3]], 'MSE':[result[4]]})
    outdf = outdf.append(testdf, ignore_index = True)

    columnsTitles = ['Station1', 'Station2', 'Phase', 'Parameter', 'Slope', 'Intercept', 'R_square', 'MSE']
outdf = outdf.reindex(columns=columnsTitles)
outdf = outdf.sort_values(by=['Parameter'])
outdf.to_csv(station1+'-'+station2+'_'+result[0]+'_correlation.csv', index=False)

In [ ]:
station1 = 'G501_COMBO3_N1'
station2 = 'G502_COMBO3_N2'
parameter = 'SFR_80IH_BL_V'
newdf=df[['Station', 'sn', parameter]]
gradient, intercept, r_value, p_value, std_err = stats.linregress(newdf[newdf['Station']==station1].groupby('sn').mean().reset_index()[parameter], 
                                                                  newdf[newdf['Station']==station2].groupby('sn').mean().reset_index()[parameter])
print('Linear regression using stats.linregress')
print('regression: a=%.2f b=%.2f, std error= %.3f' % (gradient, intercept, std_err))
p = np.poly1d(np.array([gradient, intercept]))
x = newdf[newdf['Station']==station1].groupby('sn').mean().reset_index()[parameter]
y = newdf[newdf['Station']==station2].groupby('sn').mean().reset_index()[parameter]
yhat = p(x)                         # or [p(z) for z in x]
ybar = np.sum(y)/len(y)          # or sum(y)/len(y)
ssreg = np.sum((yhat-ybar)**2)   # or sum([ (yihat - ybar)**2 for yihat in yhat])
sstot = np.sum((y - ybar)**2)
ssreg / sstot

In [ ]:
phase = 'RearSfrTestMacroCollect'
result = pd.DataFrame()
df = pd.concat([df1, df2, df3, df4, df5, df6]).reset_index()
df = df[df.columns[1:]]
for idx in range(len(fullname)):
    for idy in range(len(fullname)):
        if idx < idy:
#             print fullname[idx]+'-'+fullname[idy]+'_'+phase+'_correlation.csv'
            temp=pd.read_csv(fullname[idx]+'-'+fullname[idy]+'_'+phase+'_correlation.csv')
            result = result.append(temp, ignore_index = True)
result[(result.R_square < 0.4) & (result.Slope < .5)].sort_values(by='Parameter')
# result.sort_values(by='Parameter')
plt.hist(result.Slope)
plt.figure()
plt.hist(result.Intercept)
plt.figure()
plt.hist(result.R_square)

In [ ]:
phase = 'RearSfrTestInfCollect'
result = pd.DataFrame()
df = pd.concat([df1, df2, df3, df4, df5, df6]).reset_index()
df = df[df.columns[1:]]
for idx in range(len(fullname)):
    for idy in range(len(fullname)):
        if idx < idy:
#             print fullname[idx]+'-'+fullname[idy]+'_'+phase+'_correlation.csv'
            temp=pd.read_csv(fullname[idx]+'-'+fullname[idy]+'_'+phase+'_correlation.csv')
            result = result.append(temp, ignore_index = True)
result[(result.R_square < 0.4) & (result.Slope < .5)].sort_values(by='Parameter')
plt.hist(result.Slope)
plt.figure()
plt.hist(result.Intercept)
plt.figure()
plt.hist(result.R_square)

In [ ]:
result.head()

In [ ]:
plt.hist(result.Slope)

In [ ]:
plt.hist(result.Intercept)

In [ ]:
plt.hist(result.R_square)

In [ ]:
result[(result.R_square < 0.4) & (result.Slope < .5)].sort_values(by='Parameter')

In [ ]:
df4 = pd.read_csv(r'/home/fan/Downloads/G502_COMBO_N1/G502_COMBO_N1_phase_07_out.csv')

In [ ]:
df4

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
correlation_study(df, fullname[idx], fullname[idy], 'ov7251_slave.optical_center_y', isplot=1)


In [ ]:
station1=fullname[idx]
station2=fullname[idy]
parameter='imx520.reprojection_error_mean'
stats.linregress(newdf[newdf['Station']==station1].groupby('sn').mean().reset_index()[parameter], 
                                                                  newdf[newdf['Station']==station2].groupby('sn').mean().reset_index()[parameter])

In [21]:
df1 = pd.read_csv(r'C:\Users\fanchang\Downloads\G602_RGEO_N1_Slot1_phase_09_out.csv')
df2 = pd.read_csv(r'C:\Users\fanchang\Downloads\G602_RGEO_N1_Slot2_phase_09_out.csv')
df3 = pd.read_csv(r'C:\Users\fanchang\Downloads\G602_RGEO_N1_Slot3_phase_09_out.csv')
# df4 = pd.read_csv(r'C:\Users\fanchang\fan_python\FGEO_G602_01\FGEO_G602_01_phase_03_out.csv')
# df5 = pd.read_csv(r'C:\Users\fanchang\fan_python\FGEO_G602_02\FGEO_G602_02_phase_03_out.csv')
# df6 = pd.read_csv(r'/home/fan/Downloads/G502_COMBO3_N3/G502_COMBO3_N3_phase_12_out.csv')
# df7 = pd.read_csv(os.path.join(folder, files[6])
# df8 = pd.read_csv(os.path.join(folder, files[7]))

# reformat dataframe to drop index
# df = pd.concat([df1, df2, df3, df4, df5, df6]).reset_index()
df = pd.concat([df1, df2, df3]).reset_index()
df = df[df.columns[1:]]

# stations = [name.split('.')[0] for name in files]
stations =['G602_RGEO_N1_Slot1', 'G602_RGEO_N1_Slot2', 'G602_RGEO_N1_Slot3',]
fullname = stations
for idx in range(len(fullname)):
    for idy in range(len(fullname)):
        if idx < idy:
            print fullname[idx], fullname[idy]
            calculation(df, fullname[idx], fullname[idy])

G602_RGEO_N1_Slot1 G602_RGEO_N1_Slot2
G602_RGEO_N1_Slot1 G602_RGEO_N1_Slot3
G602_RGEO_N1_Slot2 G602_RGEO_N1_Slot3


In [27]:
# read dataframe
df1 = pd.read_csv(r'C:\Users\fanchang\fan_python\FGEO_G601_01\FGEO_G601_01_phase_03_out.csv')
df2 = pd.read_csv(r'C:\Users\fanchang\fan_python\FGEO_G601_02\FGEO_G601_02_phase_03_out.csv')
df3 = pd.read_csv(r'C:\Users\fanchang\fan_python\FGEO_G601_03\FGEO_G601_03_phase_03_out.csv')
df4 = pd.read_csv(r'C:\Users\fanchang\fan_python\FGEO_G602_01\FGEO_G602_01_phase_03_out.csv')
df5 = pd.read_csv(r'C:\Users\fanchang\fan_python\FGEO_G602_02\FGEO_G602_02_phase_03_out.csv')
# df6 = pd.read_csv(r'/home/fan/Downloads/G502_COMBO_N3/G502_COMBO_N3_phase_09_out.csv')

# reformat dataframe to drop index
df = pd.concat([df1, df2, df3, df4, df5]).reset_index()
df = df[df.columns[1:]]

fullname=['FGEO_G601_01', 'FGEO_G601_02', 'FGEO_G601_03', 'FGEO_G602_01', 'FGEO_G602_02']
# for line in ['G501', 'G502']:
#     for station in ['N1', 'N2', 'N3']:
#         fullname.append(line+'_'+camera+'_'+station)
        
# for idx in range(len(fullname)):
#     for idy in range(len(fullname)):
#         if idx < idy:
#             print fullname[idx], fullname[idy]
#             calculation(df, fullname[idx], fullname[idy])

In [20]:
print(fullname[idx], fullname[idy])

('FGEO_G601_01', 'FGEO_G601_02')


In [22]:
df.head()

,Station,sn,Operator,Phase,imx481_tele.reprojection_error_median,imx363_wide.reprojection_error_median,imx363_wide.pose_x,imx363_wide.pose_y,imx363_wide.pose_z,imx481_tele.reprojection_error_mean,...,imx481_tele.reprojection_error_samples,imx363_wide.tangential_distortion_p2,imx363_wide.radial_distortion_k1,imx481_tele.tangential_distortion_p2,imx363_wide.radial_distortion_k3,imx363_wide.radial_distortion_k2,imx481_tele.position_z,imx363_wide.reprojection_error_samples,imx481_tele.position_x,imx481_tele.position_y
0,G602_RGEO_N1_Slot1,94KAZ00C9Q,OP1,Config4_Collect_GEO_DATA,0.482229,0.535810,0,0,0,0.554670,...,614,0,0.083718,0,0.182881,-0.237436,-0.001949,1155,0.000753,0.012764
1,G602_RGEO_N1_Slot1,94KAZ00BUP,OP1,Config4_Collect_GEO_DATA,0.510877,0.547743,0,0,0,0.587681,...,611,0,0.079963,0,0.191590,-0.242286,-0.002483,1152,0.000524,0.012433
2,G602_RGEO_N1_Slot1,941AZ07400,OP1,Config4_Collect_GEO_DATA,0.472702,0.574515,0,0,0,0.543611,...,613,0,0.085887,0,0.211338,-0.255645,-0.001548,1156,0.000325,0.012828
3,G602_RGEO_N1_Slot1,948AZ08175,OP1,Config4_Collect_GEO_DATA,0.465288,0.568672,0,0,0,0.551731,...,613,0,0.080203,0,0.182093,-0.234890,-0.001414,1160,0.000149,0.013116
4,G602_RGEO_N1_Slot1,94KAZ00C1Y,OP3,Config4_Collect_GEO_DATA,0.491132,0.546167,0,0,0,0.566849,...,612,0,0.081493,0,0.183650,-0.238204,-0.001630,1150,0.000170,0.012815


In [29]:
df.Station.unique()

array(['FGEO_G601_01', 'FGEO_G601_02', 'FGEO_G601_03', 'FGEO_G602_01',
       'FGEO_G602_02'], dtype=object)

In [30]:
df.columns

Index([u'Station', u'sn', u'Operator', u'Phase',
       u'imx520.reprojection_error_mean', u'ov7251_slave.optical_center_y',
       u'ov7251_master.position_z', u'ov7251_master.position_y',
       u'ov7251_master.position_x', u'ov7251_master.reprojection_error_mean',
       u'ov7251_slave.reprojection_error_mean', u'IRtoIR.pitch',
       u'ov7251_slave.reprojection_error_samples', u'IRtoIR.roll',
       u'ov7251_slave.radial_distortion_k1',
       u'ov7251_slave.radial_distortion_k2',
       u'ov7251_slave.radial_distortion_k3', u'imx520.optical_center_y',
       u'imx520.optical_center_x', u'ov7251_slave.pose_z',
       u'ov7251_slave.tangential_distortion_p1', u'imx520.position_x',
       u'imx520.position_y', u'imx520.position_z',
       u'ov7251_slave.focal_length', u'imx520.reprojection_error_samples',
       u'imx520.pose_y', u'ov7251_master.pose_y', u'ov7251_master.pose_x',
       u'ov7251_slave.position_y', u'ov7251_master.pose_z',
       u'imx520.focal_length', u'ov7251_master

In [70]:
# read dataframe
df1 = pd.read_csv(r'C:\Users\fanchang\fan_python\C2F2_RGEO\G601_RGEO_N1_Slot1\G601RGEO01_Slot1_phase_09_out.csv')
df2 = pd.read_csv(r'C:\Users\fanchang\fan_python\C2F2_RGEO\G601_RGEO_N1_Slot2\G601RGEO01_Slot2_phase_09_out.csv')
df3 = pd.read_csv(r'C:\Users\fanchang\fan_python\C2F2_RGEO\G601_RGEO_N1_Slot3\G601RGEO01_Slot3_phase_09_out.csv')
df4 = pd.read_csv(r'C:\Users\fanchang\fan_python\C2F2_RGEO\G601_RGEO_N2_Slot1\G601_RGEO_N2_Slot1_phase_09_out.csv')
df5 = pd.read_csv(r'C:\Users\fanchang\fan_python\C2F2_RGEO\G601_RGEO_N2_Slot2\G601_RGEO_N2_Slot2_phase_09_out.csv')
df6 = pd.read_csv(r'C:\Users\fanchang\fan_python\C2F2_RGEO\G601_RGEO_N2_Slot3\G601_RGEO_N2_Slot3_phase_09_out.csv')
df7 = pd.read_csv(r'C:\Users\fanchang\fan_python\C2F2_RGEO\G602_RGEO_N1_Slot1\G602_RGEO_N1_Slot1_phase_09_out.csv')
df8 = pd.read_csv(r'C:\Users\fanchang\fan_python\C2F2_RGEO\G602_RGEO_N1_Slot2\G602_RGEO_N1_Slot2_phase_09_out.csv')
df9 = pd.read_csv(r'C:\Users\fanchang\fan_python\C2F2_RGEO\G602_RGEO_N1_Slot3\G602_RGEO_N1_Slot3_phase_09_out.csv')
df10 = pd.read_csv(r'C:\Users\fanchang\fan_python\C2F2_RGEO\G602_RGEO_N2_Slot1\G602_RGEO_N2_Slot1_phase_09_out.csv')
df11 = pd.read_csv(r'C:\Users\fanchang\fan_python\C2F2_RGEO\G602_RGEO_N2_Slot2\G602_RGEO_N2_Slot2_phase_09_out.csv')
df12 = pd.read_csv(r'C:\Users\fanchang\fan_python\C2F2_RGEO\G602_RGEO_N2_Slot3\G602_RGEO_N2_Slot3_phase_09_out.csv')
# df7 = pd.read_csv(os.path.join(folder, files[6])
# df8 = pd.read_csv(os.path.join(folder, files[7]))

# reformat dataframe to drop index
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12]).reset_index()

df = df[df.columns[1:]]

# stations = [name.split('.')[0] for name in files]
stations =['G601RGEO01_Slot1',  'G601RGEO01_Slot2', 'G601RGEO01_Slot3',
           'G601_RGEO_N2_Slot1',  'G601_RGEO_N2_Slot2', 'G601_RGEO_N2_Slot3',
           'G602_RGEO_N1_Slot1',  'G602_RGEO_N1_Slot2', 'G602_RGEO_N1_Slot3',
           'G602_RGEO_N2_Slot1',  'G602_RGEO_N2_Slot2', 'G602_RGEO_N2_Slot3',
          ]
fullname = stations
for idx in range(len(fullname)):
    for idy in range(len(fullname)):
        if idx < idy:
            print fullname[idx], fullname[idy]
            calculation(df, fullname[idx], fullname[idy])

G601RGEO01_Slot1 G601RGEO01_Slot2
G601RGEO01_Slot1 G601RGEO01_Slot3
G601RGEO01_Slot1 G601_RGEO_N2_Slot1
G601RGEO01_Slot1 G601_RGEO_N2_Slot2
G601RGEO01_Slot1 G601_RGEO_N2_Slot3
G601RGEO01_Slot1 G602_RGEO_N1_Slot1
G601RGEO01_Slot1 G602_RGEO_N1_Slot2
G601RGEO01_Slot1 G602_RGEO_N1_Slot3
G601RGEO01_Slot1 G602_RGEO_N2_Slot1
G601RGEO01_Slot1 G602_RGEO_N2_Slot2
G601RGEO01_Slot1 G602_RGEO_N2_Slot3
G601RGEO01_Slot2 G601RGEO01_Slot3
G601RGEO01_Slot2 G601_RGEO_N2_Slot1
G601RGEO01_Slot2 G601_RGEO_N2_Slot2
G601RGEO01_Slot2 G601_RGEO_N2_Slot3
G601RGEO01_Slot2 G602_RGEO_N1_Slot1
G601RGEO01_Slot2 G602_RGEO_N1_Slot2
G601RGEO01_Slot2 G602_RGEO_N1_Slot3
G601RGEO01_Slot2 G602_RGEO_N2_Slot1
G601RGEO01_Slot2 G602_RGEO_N2_Slot2
G601RGEO01_Slot2 G602_RGEO_N2_Slot3
G601RGEO01_Slot3 G601_RGEO_N2_Slot1
G601RGEO01_Slot3 G601_RGEO_N2_Slot2
G601RGEO01_Slot3 G601_RGEO_N2_Slot3
G601RGEO01_Slot3 G602_RGEO_N1_Slot1
G601RGEO01_Slot3 G602_RGEO_N1_Slot2
G601RGEO01_Slot3 G602_RGEO_N1_Slot3
G601RGEO01_Slot3 G602_RGEO_N2_Slot

In [64]:
import os
import re
filenames = os.listdir(os.getcwd())
for filename in filenames:
    if filename.endswith('.csv'):
        print filename

SyntaxError: invalid syntax (<ipython-input-64-e334f081b334>, line 1)

In [72]:
temp = pd.read_csv(r'G501_RGEO_N1_Slot1-G501_RGEO_N1_Slot2_Config1_Collect_GEO_DATA_correlation.csv')
temp = temp.dropna()
result = temp[(temp.R_square < 0.4) & (temp.Slope < .5)].sort_values(by='Parameter')
print result[['Parameter', 'Slope', 'R_square']]

                                 Parameter  Slope  R_square
13   imx363_wide.reprojection_error_median  0.257     0.146
25                  imx481_tele.position_z  0.123     0.116
31  imx481_tele.reprojection_error_samples  0.023     0.002
